# Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from torchvision import datasets, transforms
from PIL import Image
from collections import Counter
from torchvision.datasets import ImageFolder
import pandas as pd
from tqdm import tqdm
import shutil

# Add Augmented Data for Training

In [ ]:
from torchvision.transforms.functional import rotate

# Define paths
train_dir = "/kaggle/input/cards-image-datasetclassification/train"

# Define augmentation transformations
augmentations = {
    "rot_30": lambda img: rotate(img, 30, expand=True),
    "rot_90": lambda img: rotate(img, 90, expand=True)
}

# Output directory for augmented images
augmented_train_dir = "/kaggle/working/augmented_train"
if os.path.exists(augmented_train_dir):
    shutil.rmtree(augmented_train_dir)
shutil.copytree(train_dir, augmented_train_dir)

# Loop through each class directory and apply augmentations
for card_class in tqdm(os.listdir(train_dir), desc="Processing classes"):
    class_path = os.path.join(train_dir, card_class)
    augmented_class_path = os.path.join(augmented_train_dir, card_class)
    
    if not os.path.isdir(class_path):
        continue  # Skip non-directory files
    
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            continue
        
        # Apply augmentations and save them
        for aug_name, aug_func in augmentations.items():
            augmented_img = aug_func(image)
            
            augmented_img_name = f"{os.path.splitext(img_name)[0]}_{aug_name}.jpg"
            augmented_img_path = os.path.join(augmented_class_path, augmented_img_name)
            augmented_img.save(augmented_img_path, "JPEG")
            
print("Data augmentation completed. Augmented images are stored in:", augmented_train_dir)


In [ ]:
valid_dir = "/kaggle/input/cards-image-datasetclassification/valid"
test_dir = "/kaggle/input/cards-image-datasetclassification/test"

In [ ]:
batch_size = 32
image_size = (224, 224)
num_classes = 53
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ]),
    "valid": transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ]),
    "test": transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ]),
}

In [ ]:
train_dataset = ImageFolder(augmented_train_dir, transform=data_transforms["train"])
valid_dataset = ImageFolder(valid_dir, transform=data_transforms["valid"])
test_dataset = ImageFolder(test_dir, transform=data_transforms["test"])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Architecture

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, expansion=2):
        super().__init__()
        expanded_channels = in_channels * expansion
        self.conv1 = nn.Conv2d(in_channels, expanded_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(expanded_channels)
        self.conv2 = nn.Conv2d(expanded_channels, in_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)
    
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return F.relu(x)

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Initial Conv Block
        self.conv1 = nn.Conv2d(3, 64, 3, stride=2, padding=1)  # 112x112
        self.bn1 = nn.BatchNorm2d(64)
        
        # Stage 1: 112x112
        self.stage1 = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64),
            nn.MaxPool2d(2, 2)  # 56x56
        )
        
        # Stage 2: 56x56 → 28x28
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.stage2 = nn.Sequential(
            ResidualBlock(128),
            ResidualBlock(128),
            nn.MaxPool2d(2, 2)  # 28x28
        )
        
        # Stage 3: 28x28 → 14x14
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.stage3 = nn.Sequential(
            ResidualBlock(256),
            ResidualBlock(256),
            nn.MaxPool2d(2, 2)  # 14x14
        )
        
        # Head
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # 112x112
        x = self.stage1(x)  # 56x56
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.stage2(x)  # 28x28
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.stage3(x)  # 14x14
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)

In [ ]:
model = CNNModel(num_classes = num_classes).to(device)

In [ ]:
def accuracy(y_pred,y_true):
    y_pred = F.softmax(y_pred,dim = 1)
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [ ]:
class ModelTrainer():
    def __init__(self, criterion=None, optimizer=None, schedular=None, patience=6, l1_lambda=1e-5):
        self.criterion = criterion
        self.optimizer = optimizer
        self.schedular = schedular
        self.patience = patience  # Early stopping patience
        self.early_stop = False
        self.wait = 0  # Counter for patience
        self.best_valid_loss = np.Inf  # Best validation loss
        self.best_val_acc = 0
        self.best_train_acc = 0
        self.l1_lambda = l1_lambda  # L1 regularization strength

    def compute_l1_penalty(self, model):
        l1_penalty = 0.0
        for param in model.parameters():
            if param.requires_grad:
                l1_penalty += torch.sum(torch.abs(param))
        return l1_penalty

    def train_batch_loop(self, model, trainloader):
        train_loss = 0.0
        train_acc = 0.0

        for images, labels in tqdm(trainloader):
            # Move the data to device (CPU/GPU)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = self.criterion(outputs, labels)

            # Add L1 penalty to loss
            l1_penalty = self.compute_l1_penalty(model)
            loss += self.l1_lambda * l1_penalty

            # Backward pass and optimization
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            train_acc += accuracy(outputs, labels)

        return train_loss / len(trainloader), train_acc / len(trainloader)

    def valid_batch_loop(self, model, validloader):
        valid_loss = 0.0
        valid_acc = 0.0

        for images, labels in tqdm(validloader):
            # Move the data to device (CPU/GPU)
            images = images.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(images)
                loss = self.criterion(outputs, labels)

            valid_loss += loss.item()
            valid_acc += accuracy(outputs, labels)

        return valid_loss / len(validloader), valid_acc / len(validloader)

    def fit(self, model, trainloader, validloader, epochs):
        for i in range(epochs):
            if self.early_stop:
                print("Early stopping triggered. Exiting training loop.")
                break

            # Training phase
            model.train()
            avg_train_loss, avg_train_acc = self.train_batch_loop(model, trainloader)

            # Validation phase
            model.eval()
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model, validloader)

            # Check for improvement in validation loss
            if avg_valid_loss < self.best_valid_loss: 
                print(f"Validation loss decreased ({self.best_valid_loss:.6f} --> {avg_valid_loss:.6f}). Saving model...")
                torch.save(model.state_dict(), f'cnn_v10_res22k_{avg_train_acc:.2f}_{avg_valid_acc:.2f}.pt')
                self.best_valid_loss = avg_valid_loss
                self.wait = 0  # Reset patience counter
            elif avg_train_acc > self.best_train_acc and avg_valid_acc > self.best_val_acc: 
                print(f"Training accuracy increased ({self.best_train_acc:.6f} --> {avg_train_acc:.6f}).")
                print(f"Validation accuracy increased ({self.best_val_acc:.6f} --> {avg_valid_acc:.6f}). Saving model...")
                torch.save(model.state_dict(), f'cnn_v10_res22k_{avg_train_acc:.2f}_{avg_valid_acc:.2f}.pt')
                self.best_train_acc = avg_train_acc
                self.best_val_acc = avg_valid_acc
                self.wait = 0  # Reset patience counter
            else:
                self.wait += 1
                print(f"Validation loss did not improve. Patience counter: {self.wait}/{self.patience}")
                if self.wait >= self.patience:
                    print("Early stopping condition met.")
                    self.early_stop = True

            # Logging
            print(f"Epoch {i+1}/{epochs} | Train Loss: {avg_train_loss:.6f} | Train Acc: {avg_train_acc:.6f}")
            print(f"Epoch {i+1}/{epochs} | Valid Loss: {avg_valid_loss:.6f} | Valid Acc: {avg_valid_acc:.6f}")


# weighted loss

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_labels = sorted(os.listdir(augmented_train_dir))  # Assuming folder names are class labels
num_classes = len(class_labels)

# Count the number of samples per class
class_counts = {label: len(os.listdir(os.path.join(augmented_train_dir, label))) for label in class_labels}

# Convert class labels to indices
class_to_idx = {label: idx for idx, label in enumerate(class_labels)}
y_train = []

# Collect all labels from the training dataset
for label, count in class_counts.items():
    y_train.extend([class_to_idx[label]] * count)

# Compute class weights using sklearn
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)

# Convert to tensor for PyTorch loss function
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print(class_weights_tensor)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
trainer = ModelTrainer(criterion,optimizer)
trainer.fit(model, train_loader, valid_loader, epochs = 50)

# Testing 

In [ ]:
def prediction(model_dir, data_loader, model):
    model.load_state_dict(torch.load(model_dir))
    model.eval()
    loss = 0.0
    acc = 0.0
    for images,labels in tqdm(data_loader):
            
            # move the data to CPU
            images = images.to(device) 
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            
            loss += loss.item()
            acc += accuracy(outputs,labels)
    
    print("Test Loss : {:.6f} Test Acc : {:.6f}".format(loss / len(data_loader), acc / len(data_loader)))

In [ ]:
prediction("/kaggle/working/cnn_v10_res22k_0.94_0.97.pt", test_loader, model)

In [ ]:
prediction("/kaggle/working/cnn_v10_res22k_0.95_0.98.pt", test_loader, model)

In [ ]:
prediction("/kaggle/working/cnn_v10_res22k_0.96_0.98.pt", test_loader, model)